## Методические указания по выполнению лабораторной работы №3

**Тема: Обнаружение объектов с использованием Faster R-CNN**

**Цель работы:** Ознакомиться с архитектурой Faster R-CNN и принципами двухэтапного обнаружения объектов.

**Задачи:**
- Изучить теоретические основы двухэтапного обнаружения объектов: роль RPN и классификационного этапа в Faster R-CNN.
- Загрузить предобученную модель Faster R-CNN.
- Ознакомиться с форматом аннотаций для обучения в задаче детекции.
- Визуализировать предсказания, проанализировать ошибки модели и провести исследование по поиску баланса FN/FP.

### 1. Теоретическая часть

В данной лабораторной работе мы познакомимся с задачей детекции на примере архитектуры [Faster R-CNN](https://arxiv.org/pdf/1506.01497), обученной на наборе данных [COCO](https://cocodataset.org/#home), а также с новым форматом данных для обучения нейро-сетевых моделей детекции. Для оценки модели воспользуемся набором данных [Pascal VOC 2007](http://host.robots.ox.ac.uk/pascal/VOC/).

**Перед тем, как приступать к выполнению практической части, ознакомьтесь с первоисточниками используемых компонентов, а также документацией по [ссылке](https://pytorch.org/vision/master/models/faster_rcnn.html), включающей подробности работы с моделью и новым форматом данных, сэмплы кода.**

#### 1.1 Архитектура Faster R-CNN

Существует два основных подхода к обнаружению объектов:
Двухстадийные модели – более точные, но медленные.
Одностадийные модели – быстрые, но менее точные.

Faster R-CNN — это двухэтапная модель детекции объектов. В отличие от ResNeXt, Faster R-CNN не просто классифицирует изображение, а находит на нём несколько объектов, предсказывает bounding boxes и присваивает метки классам. Она состоит из следующих компонентов:

1. Backbone (ResNet/VGG/MobileNet) – извлекает признаки из изображения.
2. Region Proposal Network (RPN) – предлагает области, где могут находиться объекты.
3. ROI Pooling + Fully Connected Layers – классифицирует объекты и уточняет bounding boxes.
4. Non-Maximum Suppression (NMS) – убирает дублирующиеся предсказания.


В предыдущих лабораторных работах мы познакомились с классификацией изображений, где модель предсказывает единственный класс для всего изображения. Однако во многих задачах компьютерного зрения классификация недостаточна. Например, когда на одном изображении присутствуют несколько объектов разных классов необходимо не только определить, что изображено, но и где это находится.

Faster R-CNN –  модель, которая также решает задачу обнаружения объектов, добавляя к классификации локализацию. 

#### 1.2 Формат данных для задачи детекции

Faster R-CNN требует разметки изображений, помимо классов включающей в себя и координаты bounding box по оси x, y.
Ознакомьтесь с форматом набора данных Pascal VOC, скачайте аннотации набора данных и изучите структуру **.xml** файлов в папке Annotations по [ссылке](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtestnoimgs_06-Nov-2007.tar).

#### 1.3 Оценка качества

Помимо известных вам инструментов оценить качество работы детектора могут помочь:

Confidence Score - значение, указывающее на уверенность модели в том, что на данном месте изображения находится объект. Модель предсказывает этот параметр для каждого предсказанного bounding box. Чем выше confidence score, тем более уверена модель в своём предсказании.

Фильтрация предсказаний - модель может предсказать много объектов, но не все из них будут точными. Чтобы уменьшить количество ложных срабатываний (False Positives), применяется фильтр предсказаний, используя confidence threshold. Если confidence score меньше заданного порога - предсказание отбрасывается.


### 2. Практическая часть

#### 2.1 Подготовка окружения

Установите зависимости и библиотеки:

In [1]:
import torch
import torchvision
from torchvision import transforms, datasets
from PIL import Image, ImageDraw
import numpy as np
import xml.etree.ElementTree as ET
import os

#### 2.2 Подготовка модели

Загрузите предобученную модель, определите устройство, переведите модель в режим инференса:

In [2]:
# Загрузка предобученной модели Faster R-CNN
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

#### 2.3 Загрузка и предобработка изображений


Затем импортируйте датасет из соответствующих пакетов PyTorch и определите метод трансформации данных для подачи в модель. Он понадобится позже для преобразования изображений при прямом проходе через модель чтобы получить предсказания:

In [3]:
# Пути к датасету PASCAL VOC 2007
voc_images_path = "JPEGImages"
voc_annotations_path = "Annotations"


#### 2.4 Объявление методов для работы с данными

Далее необходимо создать методы препроцессинга: метод чтения файла аннотации для возврата numpy-объекта содержащего bounding boxes, и метод отрисовки истиных и прогнозных bounding boxes на изображении для визуализации полученных результатов.

In [4]:
# Функция для обработки аннотаций PASCAL VOC
def parse_voc_annotation(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    boxes = []
    labels = []
    
    for obj in root.findall("object"):
        label = obj.find("name").text
        bbox = obj.find("bndbox")
        xmin, ymin, xmax, ymax = map(int, [
            bbox.find("xmin").text,
            bbox.find("ymin").text,
            bbox.find("xmax").text,
            bbox.find("ymax").text
        ])
        boxes.append([xmin, ymin, xmax, ymax])
        labels.append(label)
    
    return np.array(boxes), labels

# Функция для визуализации предсказаний
def draw_boxes(image, boxes, labels, color="blue"):
    draw = ImageDraw.Draw(image)
    for box, label in zip(boxes, labels):
        xmin, ymin, xmax, ymax = map(int, box)
        draw.rectangle([xmin, ymin, xmax, ymax], outline=color, width=2)
        draw.text((xmin, ymin), f"{label}", fill=color)
    return image

#### 2.5 Анализ False Positives / False Negatives

Для измерения того, насколько хорошо bounding box предсказан, применяется параметр IoU (Intersection over Union) между двумя bounding boxes. Чем он выше, тем точнее предсказание. Для его оценки опишем следующий метод, принимающий два np.array-объекта (прогнозные и истинные координаты):

In [5]:
# Функция для вычисления IoU
def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = inter_area / (box1_area + box2_area - inter_area)
    return iou

#### 2.6 Оценка модели и визуализация результатов

Выполните прямой проход нескольких изображений через модель. Для этого необходимо загрузить изображения и аннотации, применить преобразование изображений в тензор. Затем получите выходы модели:

In [6]:
# 2.6 Оценка модели и визуализация результатов
N = 5  # Количество изображений для оценки
image_filenames = os.listdir(voc_images_path)[:N]

false_positives = 0
false_negatives = 0

for image_name in image_filenames:
    image_path = os.path.join(voc_images_path, image_name)
    annotation_path = os.path.join(voc_annotations_path, image_name.replace(".jpg", ".xml"))
    
    image = Image.open(image_path).convert("RGB")
    true_boxes, true_labels = parse_voc_annotation(annotation_path)
    
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    image_tensor = transform(image).unsqueeze(0)
    
    with torch.no_grad():
        outputs = model(image_tensor)
    
    pred_boxes = outputs[0]['boxes'].cpu().numpy()
    pred_labels = outputs[0]['labels'].cpu().numpy()
    pred_scores = outputs[0]['scores'].cpu().numpy()
    
    confidence_threshold = 0.5
    valid_indices = np.where(pred_scores > confidence_threshold)[0]
    pred_boxes = pred_boxes[valid_indices]
    pred_labels = pred_labels[valid_indices]
    pred_scores = pred_scores[valid_indices]
    
    image_with_preds = draw_boxes(image.copy(), pred_boxes, pred_labels, color="red")
    image_with_truth = draw_boxes(image.copy(), true_boxes, true_labels, color="green")
    
    image_with_preds.show(title="Predictions")
    image_with_truth.show(title="Ground Truth")

    # Вычисление IoU и подсчет FN/FP
    matched = np.zeros(len(true_boxes))
    for pred_box in pred_boxes:
        ious = [compute_iou(pred_box, true_box) for true_box in true_boxes]
        if max(ious) > 0.5:
            matched[np.argmax(ious)] = 1
        else:
            false_positives += 1
    
    false_negatives += len(true_boxes) - sum(matched)


#### 2.7 Поиск оптимальной конфигурации 

Проанализируйте полученные результаты с выбранным значением фильтрации предсказаний. Проведите исследование с целью поиска оптимального порога и баланса FN/FP. Обоснуйте полученные результаты:

In [7]:
# 2.7 Поиск оптимальной конфигурации
fp_rate = false_positives / (false_positives + false_negatives)
fn_rate = false_negatives / (false_positives + false_negatives)
print(f"False Positives: {false_positives}, False Negatives: {false_negatives}")
print(f"FP Rate: {fp_rate:.2f}, FN Rate: {fn_rate:.2f}")

# Поиск оптимального порога
best_threshold = 0.5
best_balance = abs(fp_rate - fn_rate)
for threshold in np.arange(0.3, 0.9, 0.05):
    valid_indices = np.where(pred_scores > threshold)[0]
    pred_boxes = pred_boxes[valid_indices]
    pred_labels = pred_labels[valid_indices]
    pred_scores = pred_scores[valid_indices]
    
    false_positives = 0
    false_negatives = 0
    matched = np.zeros(len(true_boxes))
    for pred_box in pred_boxes:
        ious = [compute_iou(pred_box, true_box) for true_box in true_boxes]
        if max(ious) > 0.5:
            matched[np.argmax(ious)] = 1
        else:
            false_positives += 1
    
    false_negatives += len(true_boxes) - sum(matched)
    fp_rate = false_positives / (false_positives + false_negatives)
    fn_rate = false_negatives / (false_positives + false_negatives)
    balance = abs(fp_rate - fn_rate)
    
    if balance < best_balance:
        best_balance = balance
        best_threshold = threshold

print(f"Оптимальный порог: {best_threshold}")


False Positives: 10, False Negatives: 1.0
FP Rate: 0.91, FN Rate: 0.09
Оптимальный порог: 0.5


C:\Users\User\AppData\Local\Temp\ipykernel_22120\2331076962.py:27: RuntimeWarning: invalid value encountered in scalar divide
  fp_rate = false_positives / (false_positives + false_negatives)
C:\Users\User\AppData\Local\Temp\ipykernel_22120\2331076962.py:28: RuntimeWarning: invalid value encountered in scalar divide
  fn_rate = false_negatives / (false_positives + false_negatives)
